# Ising Observables 

In this programme, we calculate the observables as functions of $T$, obtain values for $T_c$ at varying $N$ and hence find $T_c(\infty)$

## Functions

Function to Hide/Toggle cells

In [ ]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()


Importing libraries/ setting fonts

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
import imageio
import math
import pickle 
from scipy.optimize import curve_fit

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

hide_toggle()

#### lattice_same(N)
Input: $N$

Output: Lattice of size $N \times N$ where each value is $1$

In [ ]:
def lattice_same(N):
    output = np.full((N,N),1)
    return output

hide_toggle()

#### lattice_rand(N)
Input: $N$

Output: Lattice of size $N \times N$ where each value is randomly assigned $\pm1$

In [ ]:
def lattice_rand(N):
    output = (np.random.randint(0,2,size=(N,N))*2 -1)
    return output
hide_toggle()

#### E_array(T)

Input: $T$

Output: Array of shape (3,10) that contains the $E$, $\Delta E$ and $P_{flip}$ of a spin given the state of its neighbouring spins.

Each column in the array corresponds a different configuration of spins e.g. the first column gives the energy for a down spin surrounded by 4 down spins. This order is seen in Table 2 of the report

In [ ]:
def E_array(T):
    
    #energy of a spin given its nearest neighbours
    spin_E_array=([-4,4,-2,2,0,0,2,-2,4,-4])
    
    #delta_E of flipping the spin (
    delta_E_array =np.array([8,-8,4,-4,0,0,-4,4,-8,8])
    
    P_array = np.array([np.exp(-8/T),1,np.exp(-4/T),1,1,1,1,np.exp(-4/T),1,np.exp(-8/T)])
    
    
    #Forming a single array from these 3 arrays
    E_array = np.array([spin_E_array,delta_E_array,P_array])
    return E_array

hide_toggle()

#### lattice_step(lattice,T, step_no)
Input: An initial lattice, $T$ and the number of $MCs$ we calculate for

Outputs: The state of the lattice and its energy after each $MCs$

This function is implemented via a Metropolis algorithm

In [ ]:
def lattice_step(lattice,T, step_no):
    
    lattice_i=lattice.copy()
     
    #L_size is the number of points on the lattice= N^2
    L_size=np.size(lattice_i)
    
    #N is the length of side of square input lattice
    N=math.isqrt(L_size)
    
    #Creating a blank set of NxN lattices, one for each time step
    lattice_set=np.zeros((step_no+1,N,N))
    
    #Set the first lattice in this step to be the initial lattice
    lattice_set[0]=lattice_i
    
    E_array_local = E_array(T)

    #set the initial energy to zero
    E_i=0
    E_set=[]
    #Each t value corresponds to a step in time
    
    for t in range(step_no+1):
        
        #Creating a list of numbers assigned to each point on the lattice and then randomly changing the order
        
        point_list_rand=np.arange(L_size)
        np.random.shuffle(point_list_rand)
        
        
        #We then use the % and floor functions to generate a random list of points on the lattice. e.g. x_r[i], y_r[i] gives a
        #random co-ordinate on the lattice. Working through all [i] up to N^2, we cover the entire lattice
        y_r = (point_list_rand/N).astype(int)
        
        x_r = point_list_rand%N
        
        
        for i in range(L_size):
            y=np.random.rand()
        
            #we look at the surrounding neighbours of the spin    
            neighbour_sum = lattice_i[x_r[i],(y_r[i]+1)%N] + lattice_i[x_r[i],(y_r[i]-1)%N] + lattice_i[(x_r[i]+1)%N,y_r[i]] + lattice_i[(x_r[i]-1)%N,y_r[i]] + lattice_i[x_r[i],y_r[i]]/2
           
            #based on these neighbours, we pick out a column in E_array, where we obtain a value for E, delta_E 
            #and P(flip)
            array_val = int(neighbour_sum +4.5)
            
            #For the first time step, we don't flip any of the spins, we just work our way through the lattice 
            #and calculate the initial energy
            
            if t==0:
                
                #The factor of a half accounts for double counting
                E_i=0.5*E_array_local[0,array_val] +E_i
                
                
            #For all other steps, we proceed to flip the spins    
            else:
                if E_array_local[1,array_val]<=0:
                    
                    #Flip the spin
                    lattice_i[x_r[i],y_r[i]]= -lattice_i[x_r[i],y_r[i]]
                    
                    #Add delta_E to the total energy
                    E_i = E_i + E_array_local[1,array_val] 
                    
                elif E_array_local[2,array_val] > y:
                    
                    #Flip the spin
                    lattice_i[x_r[i],y_r[i]]= -lattice_i[x_r[i],y_r[i]]
                    
                    #Add delta_E to the total energy
                    E_i = E_i + E_array_local[1,array_val] 
                else:
                    
                    #Leave the spin
                    lattice_i[x_r[i],y_r[i]]= lattice_i[x_r[i],y_r[i]]
                    
                    
        lattice_set[t]=lattice_i
        E_set = np.append(E_set,E_i)
            
        
    return lattice_set,E_set


hide_toggle()

#### spin_M(lattice_set)
Input: A set of lattices at each $MCs$

Output: $m$ at each $MCs$


In [ ]:
def spin_M(lattice_set):
    M= np.sum(lattice_set,(1,2))/(np.size(lattice_set[0]))
    
    return M

hide_toggle()

#### spin_M_i(lattice_i,T,step_no)
Input: An initial lattice, $T$ and number of $MCs$

Output: $m$ at each $MCs$

In [ ]:
def spin_M_i(lattice_i,T,step_no):
    M= np.sum(lattice_step(lattice_i,T,step_no)[0],(1,2))/(np.size(lattice_i))
    
    return M

hide_toggle()

#### SHC(E,Esq,T)

Input: $\langle E \rangle , \langle E^2 \rangle, T$

Output: $C$

In [ ]:
def SHC (E,Esq,T):
    C=(Esq-E**2)/(T**2)
    
    return C

hide_toggle()

#### lattice_step_intrvl(lattice,T,n,delta_t)
Input: An initial lattice, $T$, the number of samples we wish to take and the time interval between samples

Outputs: The state of the lattice and its energy after each $MCs$

This function doesn't record the first 1000 steps so that the system reaches equilibrium 

In [ ]:
def lattice_step_intrvl(lattice,T,n,delta_t):
    
    lattice_i=lattice.copy()
     
    L_size=np.size(lattice_i)
    
    N=math.isqrt(L_size)
    
    #Creating a blank set of NxN lattices, one for step we wish to sample at
    lattice_set=np.zeros((1,N,N))
    
    
    E_array_local = E_array(T)

    #set the initial energy to zero
    E_i=0
    E_set=[]
    #Each t value corresponds to a step in time
    
    for t in range((n-1)*delta_t+1002):
        
        #Creating a list of numbers assigned to each point on the lattice and then randomly changing the order
        
        point_list_rand=np.arange(L_size)
        np.random.shuffle(point_list_rand)
        
        
        #We then use the % and floor functions to generate a random list of points on the lattice. e.g. x_r[i], y_r[i] gives a
        #random co-ordinate on the lattice. Working through all [i] up to N^2, we cover the entire lattice
        y_r = (point_list_rand/N).astype(int)
        
        x_r = point_list_rand%N
        
        
        for i in range(L_size):
            y=np.random.rand()
        
            #we look as the surrounding neighbours of the spin    
            neighbour_sum = lattice_i[x_r[i],(y_r[i]+1)%N] + lattice_i[x_r[i],(y_r[i]-1)%N] + lattice_i[(x_r[i]+1)%N,y_r[i]] + lattice_i[(x_r[i]-1)%N,y_r[i]] + lattice_i[x_r[i],y_r[i]]/2
           
            #based on these neighbours, we pick out a column in E_array, where we obtain a value for E, delta_E 
            #and P(flip)
            array_val = int(neighbour_sum +4.5)
            
            #For the first time step, we don't flip any of the spins, we just work our way through the lattice 
            #and calculate the initial energy
            
            if t==0:
                
                #The factor of a half accounts for double counting
                E_i=0.5*E_array_local[0,array_val] +E_i
                
                
            #For all other steps, we proceed to flip the spins    
            else:
                if E_array_local[1,array_val]<=0:
                    
                    #Flip the spin
                    lattice_i[x_r[i],y_r[i]]= -lattice_i[x_r[i],y_r[i]]
                    
                    #Add delta_E to the total energy
                    E_i = E_i + E_array_local[1,array_val] 
                    
                elif E_array_local[2,array_val] > y:
                    
                    #Flip the spin
                    lattice_i[x_r[i],y_r[i]]= -lattice_i[x_r[i],y_r[i]]
                    
                    #Add delta_E to the total energy
                    E_i = E_i + E_array_local[1,array_val] 
                else:
                    
                    #Leave the spin
                    lattice_i[x_r[i],y_r[i]]= lattice_i[x_r[i],y_r[i]]
       
        #Here we ignore the first 1000 values and use a mod function to ensure we only count intervals of delta_t
        if t>1000 and (t-1001)%delta_t==0:
                     
            lattice_set=np.append(lattice_set,[lattice_i],axis=0)
            E_set = np.append(E_set,E_i)
            
    
    lattice_set=lattice_set[1:]  
    return lattice_set,E_set


hide_toggle()

#### Observables(N)
Input:$N$, $T$ we sample at, a set of number of samples we want at each $T$ and a set of time intervals between samples at each $T$

Output:[$\langle E \rangle , \langle E^2 \rangle , \langle m \rangle , \langle |m| \rangle,\langle m^2 \rangle$]

In [ ]:
def observables (N,sample_T,n_set,delta_t_set):
    
    
    #creating an initially random lattice of size NxN
    lattice_i=lattice_rand(N)
    
    #We create blank arrays on the observables we want
    
    avg_M_Tset=[]
    avg_absM_Tset=[]
    avg_Msq_Tset=[]
    avg_E_Tset=[]
    avg_Esq_Tset=[]
    
    for i in range (len(sample_T)):
       

        lattice_set = lattice_step_intrvl(lattice_i,sample_T[i],n_set[i],delta_t_set[i])
        
        
        M_set=spin_M(lattice_set[0])
        
        #Average over each step
        avg_M=np.mean(M_set)
        
        #Add to our observable array at a given temperature
        avg_M_Tset=np.append(avg_M_Tset,avg_M)
        
        #We repeat this for the other observables
        Msq_set=M_set**2
        avg_Msq=np.mean(Msq_set)
        avg_Msq_Tset=np.append(avg_Msq_Tset,avg_Msq)
    
        absM_set=np.abs(M_set)
        avg_absM=np.mean(absM_set)
        avg_absM_Tset=np.append(avg_absM_Tset,avg_absM)
        
        E_set=lattice_set[1]
        avg_E=np.mean(E_set)
        avg_E_Tset=np.append(avg_E_Tset,avg_E)
        
        Esq_set=E_set**2
        avg_Esq=np.mean(Esq_set)
        avg_Esq_Tset=np.append(avg_Esq_Tset,avg_Esq)
    
        lattice_i=lattice_set[0][n_set[i]-1]
        
    return avg_M_Tset, avg_absM_Tset, avg_Msq_Tset, avg_E_Tset, avg_Esq_Tset
hide_toggle()

#### susceptibility (M,Msq,T):

Input: $\langle m \rangle, \langle m^2 \rangle, T$

Output: $\chi$

In [ ]:
def susceptibility (M,Msq,T):
    chi=(Msq-M**2)/(T)
    
    return chi

hide_toggle()

#### susceptibility_prime(absM,Msq,T)

Input: $\langle |m| \rangle,\langle m^2 \rangle, T$

Output: $\chi'$

In [ ]:
def susceptibility_prime (absM,Msq,T):
    chi=(Msq-absM**2)/(T)
    return chi
hide_toggle()

#### blocking_error(N,sample_T,n_set,delta_t_set)


Input: Lattice size, $T$ we want to sample at, the number of samples and time interval corresponding to each of these temperature

Output: $\chi$, $C$ and their erros

This function works by using a blocking method- for example $C$ is calculated in time blocks, and then finding a deviation of these blocked values gives us an estimate of the overall error

In [ ]:
def blocking_error (N,sample_T,n_set,delta_t_set):
    
    
    #creating an initially random lattice of size NxN
    lattice_i=lattice_rand(N)
    
    #We create blank arrays on the observables we want
    sus_total_set=[]
    shc_total_set=[]
    sus_err_set=[]
    shc_err_set=[]
    
    for i in range (len(sample_T)):
        sus_set=[]
        shc_set=[]
        
        lattice_set = lattice_step_intrvl(lattice_i,sample_T[i],n_set[i],delta_t_set[i])
        
        M_set=spin_M(lattice_set[0])
        

        #We repeat this for the other observables
        Msq_set=M_set**2    
        absM_set=np.abs(M_set)  
        
        E_set=lattice_set[1]
        Esq_set=E_set**2
        
        sus_total_set=np.append(sus_total_set,susceptibility_prime(np.mean(absM_set),np.mean(Msq_set),sample_T[i])*(N**2))
        shc_total_set=np.append(shc_total_set,SHC( np.mean(E_set),np.mean(Esq_set),sample_T[i])/(N**2))
                                
        for j in range(int(n_set[i]/100)):
            sus= susceptibility_prime(np.mean(absM_set[j*100:(j+1)*100-1]),np.mean(Msq_set[j*100:(j+1)*100-1]),sample_T[i])*(N**2)
            sus_set=np.append(sus_set,sus)
            shc=SHC( np.mean(E_set[j*100:(j+1)*100-1]),np.mean(Esq_set[j*100:(j+1)*100-1]),sample_T[i])/(N**2)
            shc_set=np.append(shc_set,shc)
            
        sus_err_set= np.append(sus_err_set,np.std(sus_set,ddof=1))
        shc_err_set= np.append(shc_err_set,np.std(shc_set,ddof=1))  
                               
                               
        lattice_i=lattice_set[0][n_set[i]-1]
                                   
    return [sus_total_set, shc_total_set, sus_err_set,shc_err_set]
hide_toggle()

# Programme

Generating a set of point for a 10x10 lattice, at t=0.1, over 100 steps

In [ ]:
lattice_set_test= lattice_step(lattice_rand(10),0.1,100)[0]

Calculating the magnetisation at each mcs

In [ ]:
spin_M_test=spin_M(lattice_set_test)


$m$ for a given mcs ($N=10$, $T=0.1$)

In [ ]:
plt.rcParams['figure.figsize'] = [14,7]
plt.rcParams.update({'font.size': 22})
fig,ax=plt.subplots()

ax.plot(spin_M_test,label=("T=0.1"))

plt.xlabel("$MCs$")
plt.ylabel("$m$")


We then create a plot of the lattice at each mcs and save our results a a jpeg
The red squares correspond to $\uparrow$ spins (+1), whilst the blue squares correspond to $\downarrow$ spins (-1),  

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
images=[]
for i in range(50):

    X, Y = np.meshgrid(range(math.isqrt(np.size(lattice_set_test[0]))+1), range(math.isqrt(np.size(lattice_set_test[0]))+1))


    plt.pcolormesh(X, Y, lattice_set_test[i], cmap=plt.cm.RdBu)
    
    plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images/Sandwich/' + str(i) + '.jpeg',bbox_inches='tight')
    
    
    images.append(imageio.imread(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images/Sandwich/' + str(i) + '.jpeg'))



In [ ]:
imageio.mimsave(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images/Sandwich/Form_sandwich.gif', images,format='GIF',duration=0.25)

Plotting the analytical result for $m$ as a function of $T$ as $N \rightarrow \infty$ :

$m=[1-\sinh^{-4}(\frac{2}{T})]^{\frac{1}{8}}$

In [ ]:
end=2/np.log(1+np.sqrt(2))
temp=np.linspace(0.1,end,10000)
temp_after=np.linspace(end,5,1000)
zero=np.zeros(1000)

plt.plot(temp,(np.abs(1-(np.sinh(2/temp)**-4)))**(1/8),color='red')
plt.plot(temp_after,zero,color='red')

plt.xlabel("$T$ $(J/k_B)$")
plt.ylabel("$m$")
#plt.title("Analytical result for $m$ as a function of $T$ as $N \\rightarrow \\infty$")
plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\Analytical M.jpeg',bbox_inches='tight')

We now find the observables $\langle E \rangle , \langle E^2 \rangle , \langle m \rangle , \langle |m| \rangle$ and $\langle m^2 \rangle$ as function of temperature for lattice sizes $N=2,5,10,20$

In [ ]:
#We sample 80 points in the temperature range T=0.5 - 4.5
sample_T=np.linspace(4.5,0.5,80)

In [ ]:
n_set=np.ones(80,dtype=np.int8)*10000

In [ ]:
delta_t_set=np.concatenate((np.ones(35,dtype=np.int8),np.ones(20,dtype=np.int8)*5,np.ones(25,dtype=np.int8)))

In [ ]:
observables_2=observables(2,sample_T,n_set,delta_t_set)

In [ ]:
observables_5=observables(5,sample_T,n_set,delta_t_set)

In [ ]:
observables_10=observables(10,sample_T,n_set,delta_t_set)

In [ ]:
observables_20=observables(20,sample_T,n_set,delta_t_set)

In [ ]:
def E_2_theory(T):
    E= -(8*np.sinh(8/T))/(3 + np.cosh(8/T))
    
    return E

$\langle E \rangle$ vs $T$ $(N=2)$ compared to exact result

In [ ]:
plt.plot(sample_T,E_2_theory(sample_T), label='Theoretical',color='r',linewidth=0.5)
plt.plot(sample_T,observables_2[3],label='Simulation',color='b',marker='s',markerfacecolor='none',ls='none', linewidth=0.5, mew=0.5 )
plt.xlabel("$T$")
plt.ylabel("$\\langle E \\rangle$")
#plt.title("$\\langle E \\rangle$ vs $T$ $(N=2)$")
plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.7))
plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\EvsT theory (N=2).jpeg',bbox_inches='tight')

In [ ]:
def avg_absM_2_theory (T):
    avg_absM = (2*np.exp(8/T)+4)/(3+np.cosh(8/T))/4
    
    return avg_absM

$\langle |m| \rangle$ vs $T$ $(N=2)$, compared to exact results

In [ ]:
plt.plot(sample_T,avg_absM_2_theory(sample_T), label='Theoretical',color='r',linewidth=0.5)
plt.plot(sample_T,observables_2[1],label='Simulation',color='b',marker='x',ls='none', linewidth=0.5, mew=0.5 )
plt.xlabel("$T$")
plt.ylabel("$\\langle |m|\\rangle$")
#plt.title("$\\langle |m| \\rangle$ vs $T$ $(N=2)$")
plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.7))
plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\absMvsT comparison (N=2).jpeg',bbox_inches='tight')

$|\langle m \rangle|$ vs $T$ $(N=5)$, compared with theoretical result as $N \rightarrow \infty$

In [ ]:

plt.plot(temp,(np.abs(1-(np.sinh(2/temp)**-4)))**(1/8),color='red',linewidth=0.5)
plt.plot(temp_after,zero,color='red',linewidth=0.5)

plt.plot(sample_T,np.abs(observables_5[0]),label='$N=5$',color='c',marker='+',ls='none', linewidth=0.5, mew=1)

plt.legend(loc="upper right", bbox_to_anchor=(1,1))

plt.xlabel("$T$")
plt.ylabel("$|\\langle m \\rangle|$")
plt.xlim(0.5,4.5)
#plt.title("$|\\langle m \\rangle|$ vs $T$ $(N=5)$")


plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\abs_avgM compare_5.jpeg',bbox_inches='tight')

$|\langle m \rangle|$ vs $T$ $(N=2)$, compared with theoretical result as $N \rightarrow \infty$

In [ ]:

plt.plot(temp,(np.abs(1-(np.sinh(2/temp)**-4)))**(1/8),color='red',linewidth=0.5)
plt.plot(temp_after,zero,color='red',linewidth=0.5)

plt.plot(sample_T,np.abs(observables_2[0]),label='$N=2$',color='b',marker='+',ls='none', linewidth=0.5, mew=1)

plt.legend(loc="upper right", bbox_to_anchor=(1,1))

plt.xlabel("$T$")
plt.ylabel("$|\\langle m \\rangle|$")
plt.xlim(0.5,4.5)
#plt.title("$|\\langle m \\rangle|$ vs $T$ $(N=2)$")


plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\abs_avgM compare_2.jpeg',bbox_inches='tight')

Spontaneous Magnetisation at $N=2$, $T=0.8$

In [ ]:
plt.plot(spin_M_i(lattice_rand(2),0.8,10000))
plt.xlim([1000,10000])
plt.xlabel("$MCs$")
plt.ylabel("$m$")
#plt.title("Spontaneous Magnetisation at $N=2$, $T=0.8$")


plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\spontaneous M.jpeg',bbox_inches='tight')

In [ ]:
sample_T_N40=np.concatenate((np.linspace(3.5,2.6,10),np.linspace(2.5,2,21),np.linspace(1.9,1,9)))

In [ ]:
n_set_N40=np.concatenate((np.ones(10,dtype=np.int8)*5000,np.ones(21,dtype=np.int8)*10000,np.ones(9,dtype=np.int8)*5000))

In [ ]:
delta_t_N40=np.concatenate((np.ones(10,dtype=np.int8),np.ones(21,dtype=np.int8)*5,np.ones(9,dtype=np.int8)))

In [ ]:
observables_40=observables(40,sample_T_N40,n_set_N40,delta_t_N40)

In [ ]:
np.save('observables_40',observables_40)

$|\langle m \rangle|$ vs $T$ $(N=40)$, compared with theoretical result as $N \rightarrow \infty$

In [ ]:

plt.plot(temp,(np.abs(1-(np.sinh(2/temp)**-4)))**(1/8),color='red',linewidth=0.5)
plt.plot(temp_after,zero,color='red',linewidth=0.5)

plt.plot(sample_T_N40,np.abs(observables_40[0]),label='$N=40$',color='k',marker='+',ls='none', linewidth=0.5, mew=1 )

plt.legend(loc="upper right", bbox_to_anchor=(1,1))
plt.xlabel("$T$")
plt.ylabel("$|\\langle m \\rangle|$")
plt.xlim(0.5,4.5)
#plt.title("$|\\langle m \\rangle|$ vs $T$ $(N=20)$")


plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\abs_avgM compare_40.jpeg',bbox_inches='tight')

$\chi'$ per spin vs Temperature for varying $N$

In [ ]:
plt.plot(sample_T,susceptibility_prime(observables_5[1],observables_5[2],sample_T)*25,marker='^',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='c')
plt.plot(sample_T,susceptibility_prime(observables_10[1],observables_10[2],sample_T)*100,marker='^',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=10$',color='g')
plt.plot(sample_T,susceptibility_prime(observables_20[1],observables_20[2],sample_T)*400,marker='^',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=20$',color='m')
plt.plot(sample_T_N40,susceptibility_prime(observables_40[1],observables_40[2],sample_T_N40)*1600,marker='^',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=40$',color='k')
plt.xlabel("$T$")
plt.ylabel("$\\chi \\prime$ per spin")
#plt.title("$\\langle \\chi \\rangle$ per spin vs Temperature")
plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.5))

plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\susceptibility prime vs T.jpeg',bbox_inches='tight')

$\langle |m| \rangle$ vs $T$ for varying $N$

In [ ]:
plt.plot(sample_T,observables_2[1],marker='x',ls='-', linewidth=0.5, mew=0.5,label='N=2',color='blue')
plt.plot(sample_T,observables_5[1],marker='x',ls='-', linewidth=0.5, mew=0.5,label='N=5',color='c')
plt.plot(sample_T,observables_10[1],marker='x',ls='-', linewidth=0.5, mew=0.5,label='N=10',color='g')
plt.plot(sample_T,observables_20[1],marker='x',ls='-', linewidth=0.5, mew=0.5,label='N=20',color='m')
plt.plot(sample_T_N40,observables_40[1],marker='x',ls='-', linewidth=0.5, mew=0.5,label='N=40',color='k')
plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.4))

plt.xlabel("$T$")
plt.ylabel("$\\langle |m| \\rangle$")
#plt.title("$\\langle |m| \\rangle$ vs $T$")
plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\avgabsMvsT.jpeg',bbox_inches='tight')

$\chi$ per spin vs Temperature for varying $N$

In [ ]:
plt.plot(sample_T,susceptibility(observables_2[0],observables_2[2],sample_T)*4,marker='p',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=2$',color='b')
plt.plot(sample_T,susceptibility(observables_5[0],observables_5[2],sample_T)*25,marker='p',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='c')
plt.plot(sample_T,susceptibility(observables_10[0],observables_10[2],sample_T)*100,marker='p',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=10$',color='g')
plt.plot(sample_T,susceptibility(observables_20[0],observables_20[2],sample_T)*400,marker='p',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=20$',color='m')
plt.plot(sample_T_N40,susceptibility(observables_40[0],observables_40[2],sample_T_N40)*1600,marker='p',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=40$',color='k')

plt.xlabel("$T$")
plt.ylabel("$\\chi $ per spin")

#plt.title("$\\langle \\chi \\rangle$ per spin vs Temperature")
plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.5))
plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\susceptibility vs T.jpeg',bbox_inches='tight')

$C $ per spin vs $T$ for varying $N$

In [ ]:
plt.plot(sample_T,SHC(observables_5[3],observables_5[4],sample_T)/25,marker='o',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='c')
plt.plot(sample_T,SHC(observables_10[3],observables_10[4],sample_T)/100,marker='o',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5,label='$N=10$',color='g')
plt.plot(sample_T,SHC(observables_20[3],observables_20[4],sample_T)/400,marker='o',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5,label='$N=20$',color='m')
plt.plot(sample_T_N40,SHC(observables_40[3],observables_40[4],sample_T_N40)/1600,marker='o',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5,label='$N=40$',color='k')

plt.xlabel("$T$")
plt.ylabel("$C$ per spin")
#plt.title("$\\langle C \\rangle$ per spin vs $T$")
plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.5))

plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\SHC vs T.jpeg',bbox_inches='tight')

$\langle E \rangle$ per spin vs $T$ for varying $N$

In [ ]:
plt.plot(sample_T,observables_2[3]/4,label='N=2',color='b',marker='s',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5)
plt.plot(sample_T,observables_5[3]/25,label='N=5',color='c',marker='s',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5)
plt.plot(sample_T,observables_10[3]/100,label='N=10',color='g',marker='s',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5)
plt.plot(sample_T_N40,observables_40[3]/1600,label='N=40',color='k',marker='s',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5)

plt.plot

plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.6))

plt.xlabel("$T$")
plt.ylabel("$\\langle E \\rangle$ per spin")
#plt.title("$\\langle E \\rangle$ per spin vs $T$")
plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\EvsT per spin.jpeg',bbox_inches='tight')

We then find $T_c$ by plotting a parabola about the peak. We obtain the errors using a blocking method

In [ ]:
def parabola(x,a,b,c):
    return a*x**2 + b*x +c

$N=5$

In [ ]:
SHC_N5=SHC(observables_5[3],observables_5[4],sample_T)/25

In [ ]:
sus_N5=susceptibility_prime(observables_5[1],observables_5[2],sample_T)*25

In [ ]:
np.argmax(sus_N5)

In [ ]:
np.argmax(SHC_N5)

In [ ]:
b_err_5=blocking_error (5,sample_T[34:44],np.ones(10,dtype=np.int8)*10000,np.ones(10,dtype=np.int8)*5)

In [ ]:
plt.errorbar(sample_T[34:44],b_err_5[0],yerr=b_err_5[2],ls='None',lw=0.5, mew=0.5, marker='^', color='c',capsize=6)
plt.xlabel("$T$")
plt.ylabel("$\\chi \\prime$ per spin")

In [ ]:
p_sus5, pcov_sus5 = curve_fit(parabola, sample_T[34:44],b_err_5[0],sigma=b_err_5[2] )

In [ ]:
print(p_sus5)
perr_sus5 = np.sqrt(np.diag(pcov_sus5))
print(perr_sus5)

In [ ]:
plt.plot(sample_T,parabola(sample_T,p_sus5[0],p_sus5[1],p_sus5[2]),color='orange')
plt.plot(sample_T,sus_N5,marker='^',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='c')
plt.ylim(0,1)
plt.errorbar(sample_T[34:44],b_err_5[0],yerr=b_err_5[2],ls='None',lw=0.5, mew=0.5, marker='^', color='c',capsize=6)


In [ ]:
Tc_sus5=-(p_sus5[1]/(2*p_sus5[0]))
print(Tc_sus5)

In [ ]:
Tc_sus5_err=(np.sqrt((perr_sus5[0]/p_sus5[0])**2 + (perr_sus5[1]/p_sus5[1])**2))*Tc_sus5
print(Tc_sus5_err)

In [ ]:
plt.errorbar(sample_T[34:44],b_err_5[1],yerr=b_err_5[3],ls='None',lw=0.5, mew=0.5, marker='o', color='c',capsize=6)
plt.xlabel("$T$")
plt.ylabel("$C$ per spin")
plt.ylim(0,2)
plt.xlim(0.5,4.5)

In [ ]:
p_shc5, pcov_shc5 = curve_fit(parabola, sample_T[34:44],b_err_5[1],sigma=b_err_5[3] )

In [ ]:
print(p_shc5)
perr_shc5 = np.sqrt(np.diag(pcov_shc5))
print(perr_shc5)

In [ ]:
plt.plot(sample_T,parabola(sample_T,p_shc5[0],p_shc5[1],p_shc5[2]),color='orange')
plt.plot(sample_T,SHC_N5,marker='o',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='c')
plt.ylim(0,1)
plt.errorbar(sample_T[34:44],b_err_5[1],yerr=b_err_5[3],ls='None',lw=0.5, mew=0.5, marker='o', color='c',capsize=6)

In [ ]:
Tc_shc5=-(p_shc5[1]/(2*p_shc5[0]))
print(Tc_shc5)

In [ ]:
Tc_shc5_err=(np.sqrt((perr_shc5[0]/p_shc5[0])**2 + (perr_shc5[1]/p_shc5[1])**2))*Tc_shc5
print(Tc_shc5_err)

$N=10$

In [ ]:
SHC_N10=SHC(observables_10[3],observables_10[4],sample_T)/100

In [ ]:
sus_N10=susceptibility_prime(observables_10[1],observables_10[2],sample_T)*100

In [ ]:
np.argmax(sus_N10)

In [ ]:
np.argmax(SHC_N10)

In [ ]:
b_err_10=blocking_error (10,sample_T[39:45],np.ones(6,dtype=np.int8)*10000,np.ones(6,dtype=np.int8)*5)

In [ ]:
plt.errorbar(sample_T[39:45],b_err_10[0],yerr=b_err_10[2],ls='None',lw=0.5, mew=0.5, marker='^', color='g',capsize=6)
plt.xlabel("$T$")
plt.ylabel("$\\chi \\prime$ per spin")

In [ ]:
p_sus10, pcov_sus10 = curve_fit(parabola, sample_T[39:45],b_err_10[0],sigma=b_err_10[2] )

In [ ]:
print(p_sus10)
perr_sus10 = np.sqrt(np.diag(pcov_sus10))
print(perr_sus10)

In [ ]:
plt.plot(sample_T,parabola(sample_T,p_sus10[0],p_sus10[1],p_sus10[2]),color='orange')
plt.plot(sample_T,sus_N10,marker='^',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,color='g')
plt.ylim(0,3.5)
plt.xlim(1,4)
plt.errorbar(sample_T[39:45],b_err_10[0],yerr=b_err_10[2],ls='None',lw=0.5, mew=0.5, marker='^', color='g',capsize=6,label='$N=5$')
plt.xlabel("$T$")
plt.ylabel("$\\chi \\prime$ per spin")
plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.5))
plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\Sus_parabola_5.jpeg',bbox_inches='tight')

In [ ]:
Tc_sus10=-(p_sus10[1]/(2*p_sus10[0]))
print(Tc_sus10)

In [ ]:
Tc_sus10_err=(np.sqrt((perr_sus10[0]/p_sus10[0])**2 + (perr_sus10[1]/p_sus10[1])**2))*Tc_sus10
print(Tc_sus10_err)

In [ ]:
plt.errorbar(sample_T[39:45],b_err_10[1],yerr=b_err_10[3],ls='None',lw=0.5, mew=0.5, marker='o', color='g',capsize=6)
plt.xlabel("$T$")
plt.ylabel("$C$ per spin")
plt.ylim(0,2)
plt.xlim(0.5,4.5)

In [ ]:
p_shc10, pcov_shc10 = curve_fit(parabola, sample_T[39:45],b_err_10[1],sigma=b_err_10[3] )

In [ ]:
print(p_shc10)
perr_shc10 = np.sqrt(np.diag(pcov_shc10))
print(perr_shc10)

In [ ]:
plt.plot(sample_T,parabola(sample_T,p_shc10[0],p_shc10[1],p_shc10[2]),color='orange')
plt.plot(sample_T,SHC_N10,marker='o',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='g')
plt.ylim(0,2)
plt.errorbar(sample_T[39:45],b_err_10[1],yerr=b_err_10[3],ls='None',lw=0.5, mew=0.5, marker='o', color='g',capsize=6)


In [ ]:
Tc_shc10=-(p_shc10[1]/(2*p_shc10[0]))
print(Tc_shc10)

In [ ]:
Tc_shc10_err=(np.sqrt((perr_shc10[0]/p_shc10[0])**2 + (perr_shc10[1]/p_shc10[1])**2))*Tc_shc10
print(Tc_shc10_err)

$N=20$

In [ ]:
SHC_N20=SHC(observables_20[3],observables_20[4],sample_T)/400

In [ ]:
sus_N20=susceptibility_prime(observables_20[1],observables_20[2],sample_T)*400

In [ ]:
np.argmax(sus_N20)

In [ ]:
np.argmax(SHC_N20)

In [ ]:
b_err_20=blocking_error (20,sample_T[40:46],np.ones(6,dtype=np.int8)*10000,np.ones(6,dtype=np.int8)*5)

In [ ]:
plt.errorbar(sample_T[40:46],b_err_20[0],yerr=b_err_20[2],ls='None',lw=0.5, mew=0.5, marker='^', color='m',capsize=6)
plt.xlabel("$T$")
plt.ylabel("$\\chi \\prime$ per spin")
plt.ylim(0,12)
plt.xlim(0.5,4.5)

In [ ]:
p_sus20, pcov_sus20 = curve_fit(parabola, sample_T[40:46],b_err_20[0],sigma=b_err_20[2] )

In [ ]:
print(p_sus20)
perr_sus20 = np.sqrt(np.diag(pcov_sus20))
print(perr_sus20)

In [ ]:
plt.plot(sample_T,parabola(sample_T,p_sus20[0],p_sus20[1],p_sus20[2]),color='orange')
plt.plot(sample_T,sus_N20,marker='^',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='m')
plt.ylim(0,12)
plt.errorbar(sample_T[40:46],b_err_20[0],yerr=b_err_20[2],ls='None',lw=0.5, mew=0.5, marker='^', color='m',capsize=6)


In [ ]:
Tc_sus20=-(p_sus20[1]/(2*p_sus20[0]))
print(Tc_sus20)

In [ ]:
Tc_sus20_err=(np.sqrt((perr_sus20[0]/p_sus20[0])**2 + (perr_sus20[1]/p_sus20[1])**2))*Tc_sus20
print(Tc_sus20_err)

In [ ]:
plt.errorbar(sample_T[40:46],b_err_20[1],yerr=b_err_20[3],ls='None',lw=0.5, mew=0.5, marker='o', color='m',capsize=6)
plt.xlabel("$T$")
plt.ylabel("$C$ per spin")
plt.ylim(0,2.5)
plt.xlim(0.5,4.5)

In [ ]:
p_shc20, pcov_shc20 = curve_fit(parabola, sample_T[40:46],b_err_20[1],sigma=b_err_20[3] )

In [ ]:
print(p_shc20)
perr_shc20 = np.sqrt(np.diag(pcov_shc20))
print(perr_shc20)

In [ ]:
plt.plot(sample_T,parabola(sample_T,p_shc20[0],p_shc20[1],p_shc20[2]),color='orange')
plt.plot(sample_T,SHC_N20,marker='o',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='m')
plt.ylim(0,2)
plt.errorbar(sample_T[40:46],b_err_20[1],yerr=b_err_20[3],ls='None',lw=0.5, mew=0.5, marker='o', color='m',capsize=6)


In [ ]:
Tc_shc20=-(p_shc20[1]/(2*p_shc20[0]))
print(Tc_shc20)

In [ ]:
Tc_shc20_err=(np.sqrt((perr_shc20[0]/p_shc20[0])**2 + (perr_shc20[1]/p_shc20[1])**2))*Tc_shc20
print(Tc_shc20_err)

$N=40$

In [ ]:
SHC_N40=SHC(observables_40[3],observables_40[4],sample_T_N40)/1600

In [ ]:
sus_N40=susceptibility_prime(observables_40[1],observables_40[2],sample_T_N40)*1600

In [ ]:
np.argmax(sus_N40)

In [ ]:
np.argmax(SHC_N40)

In [ ]:
b_err_40=blocking_error (40,sample_T_N40[15:22],np.ones(7,dtype=np.int8)*10000,np.ones(7,dtype=np.int8)*1)

In [ ]:
plt.errorbar(sample_T_N40[15:22],b_err_40[0],yerr=b_err_40[2],ls='None',lw=0.5, mew=0.5, marker='^', color='k',capsize=6)
plt.xlabel("$T$")
plt.ylabel("$\\chi \\prime$ per spin")
plt.ylim(0,40)
plt.xlim(0.5,4.5)

In [ ]:
p_sus40, pcov_sus40 = curve_fit(parabola, sample_T_N40[15:22],b_err_40[0],sigma=b_err_40[2] )

In [ ]:
print(p_sus40)
perr_sus40 = np.sqrt(np.diag(pcov_sus40))
print(perr_sus40)

In [ ]:
plt.plot(sample_T_N40,parabola(sample_T_N40,p_sus40[0],p_sus40[1],p_sus40[2]),color='orange')
plt.plot(sample_T_N40,sus_N40,marker='^',markerfacecolor='None',ls='-', linewidth=0.5, mew=0.5,label='$N=5$',color='k')
plt.ylim(0,40)
plt.errorbar(sample_T_N40[15:22],b_err_40[0],yerr=b_err_40[2],ls='None',lw=0.5, mew=0.5, marker='^', color='k',capsize=6)


In [ ]:
Tc_sus40=-(p_sus40[1]/(2*p_sus40[0]))
print(Tc_sus40)

In [ ]:
Tc_sus40_err=(np.sqrt((perr_sus40[0]/p_sus40[0])**2 + (perr_sus40[1]/p_sus40[1])**2))*Tc_sus40
print(Tc_sus40_err)

In [ ]:
plt.errorbar(sample_T_N40[15:22],b_err_40[1],yerr=b_err_40[3],ls='None',lw=0.5, mew=0.5, marker='o', color='k',capsize=6)
plt.xlabel("$T$")
plt.ylabel("$C$ per spin")
plt.ylim(0,2.5)
plt.xlim(0.5,4.5)

In [ ]:
p_shc40, pcov_shc40 = curve_fit(parabola, sample_T_N40[15:22],b_err_40[1],sigma=b_err_40[3] )

In [ ]:
print(p_shc40)
perr_shc40 = np.sqrt(np.diag(pcov_shc40))
print(perr_shc40)

In [ ]:
plt.plot(sample_T_N40,parabola(sample_T_N40,p_shc40[0],p_shc40[1],p_shc40[2]),color='orange')
plt.plot(sample_T_N40,SHC_N40,marker='o',markerfacecolor='none',ls='-', linewidth=0.5, mew=0.5,color='k')
plt.ylim(0,2.5)
plt.xlim(1.5,3)
plt.xlabel("$T$")
plt.ylabel("$C$ per spin")

plt.errorbar(sample_T_N40[15:22],b_err_40[1],yerr=b_err_40[3],ls='None',lw=0.5, mew=0.5, marker='o', color='k',capsize=6,label='$N=40$')
plt.legend(loc="lower left", bbox_to_anchor=(-0, 0.5))

plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\SHC_parabola_40.jpeg',bbox_inches='tight')

In [ ]:
Tc_shc40=-(p_shc40[1]/(2*p_shc40[0]))
print(Tc_shc40)

In [ ]:
Tc_shc40_err=(np.sqrt((perr_shc40[0]/p_shc40[0])**2 + (perr_shc40[1]/p_shc40[1])**2))*Tc_shc40
print(Tc_shc40_err)

We then plot $T_c(N)$ against $N^{-1}$

In [ ]:
Tc_sus_set=np.array([Tc_sus5,Tc_sus10,Tc_sus20,Tc_sus40])

In [ ]:
Tc_sus_err=np.array([Tc_sus5_err,Tc_sus10_err,Tc_sus20_err,Tc_sus40_err])

In [ ]:
Tc_shc_set=np.array([Tc_shc5,Tc_shc10,Tc_shc20,Tc_shc40])

In [ ]:
Tc_shc_err=np.array([Tc_shc5_err,Tc_shc10_err,Tc_shc20_err,Tc_shc40_err])

In [ ]:
Tc_set=(Tc_sus_set + Tc_shc_set)/2

In [ ]:
Tc_err=np.sqrt(Tc_sus_err**2 + Tc_shc_err**2)/2

In [ ]:
plt.errorbar([0.2,0.1,0.05,0.025],Tc_sus_set,ls='None',lw=0.5, mew=0.5, marker='d', color='red',capsize=6)
plt.errorbar([0.2,0.1,0.05,0.025],Tc_shc_set,ls='None',lw=0.5, mew=0.5, marker='d', color='blue',capsize=6,)
plt.errorbar([0.2,0.1,0.05,0.025],Tc_set,yerr=Tc_err,ls='None',lw=0.5, mew=0.5, marker='d', color='k',capsize=6,)
plt.plot([0,0.21],[(p_Tc[1]),p_Tc[0]*0.21+p_Tc[1]],color='k', lw=0.5,ls='--')
plt.xlabel("$N^{-1/\\nu}$")
plt.ylabel("$T_c(N)$")
plt.xlim(0,0.21)

plt.savefig(r'C:\Users\Lucas\Documents\Third Year\Computing Project\Images\Tc vs N.jpeg',bbox_inches='tight')

In [ ]:
def linear(x,a,b):
    return a*x +b

In [ ]:
p_Tc, pcov_Tc = curve_fit(linear, [0.2,0.1,0.05,0.025],Tc_set,sigma=Tc_err )

In [ ]:
pcov_Tc

In [ ]:
print(p_Tc)

In [ ]:
perr_Tc = np.sqrt(np.diag(pcov_Tc))
print(perr_Tc)

In [ ]:
p_Tc_sus, pcov_Tc_sus = curve_fit(linear, [0.2,0.1,0.05,0.025],Tc_sus_set,sigma=Tc_sus_err )

In [ ]:
print(p_Tc_sus)

In [ ]:
perr_Tc_sus = np.sqrt(np.diag(pcov_Tc_sus))
print(perr_Tc_sus)

In [ ]:
p_Tc_shc, pcov_Tc_shc = curve_fit(linear, [0.2,0.1,0.05,0.025],Tc_shc_set,sigma=Tc_shc_err )

In [ ]:
print(p_Tc_shc)

In [ ]:
perr_Tc_shc = np.sqrt(np.diag(pcov_Tc_shc))
print(perr_Tc_shc)